Naar analogie met de exercitie voor 7 aaneensluitende zittingsdagen, hieronder een poging hetzelfde te doen voor een DBNL TEI document. Overeenkomst: leaf text elementen (in dit geval paragrafen, of wellicht tokens - begin met paragrafen) in sequentie, met daarop een hierarchische 'fysische' structuur.

In [1]:
import lxml
from lxml import etree

path = '../data/tei-samples/bosb002graa04_01.xml'

_last_page_begin_index = 0
_last_section_begin_index = -1
_last_chapter_begin_index = -1
_last_paragraph_begin_index = -1
_last_head_begin_index = -1

_last_page_end_index = -1
_last_section_end_index = -1
_last_chapter_end_index = -1
_last_paragraph_end_index = -1
_last_head_end_index = -1

_last_page_id = ""

all_textelements=[]
all_annotations=[]

# handle each of the elements in the hierarchy according to 'layer type'
def handle_element(action,e):  
    global _last_page_begin_index
    global _last_section_begin_index
    global _last_chapter_begin_index
    global _last_paragraph_begin_index
    global _last_head_begin_index

    global _last_page_end_index
    global _last_section_end_index
    global _last_chapter_end_index
    global _last_paragraph_end_index
    global _last_head_end_index
    
    global _last_page_id

    if action == 'start':
        # store last begin indexes
        if e.tag == 'p':
            _last_paragraph_begin_index = len(all_textelements)               
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_begin_index = len(all_textelements)
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_begin_index = len(all_textelements)
        elif e.tag == 'head':
            _last_head_begin_index = len(all_textelements)
    elif action == 'end':
        if e.tag == 'p': 
            # leaf text element, add to all_textelements, also include text after possible pb's
            for index, t in enumerate(e.itertext()):
                all_textelements.append(t.strip())
                if index > 0: # assume: caused by pb contained within p. Update page end.
                    _last_page_end_index = len(all_textelements)-1
                    
            _last_paragraph_end_index = len(all_textelements)-1
            all_annotations.append({'label':'paragraph','begin_index': _last_paragraph_begin_index,\
                            'end_index':_last_paragraph_end_index,'id': "generate_later"})
        elif e.tag == 'head':
            # leaf text element, add to all_textelements
            all_textelements.append(e.text)
            
            _last_head_end_index = len(all_textelements)-1
            all_annotations.append({'label':'head','begin_index': _last_head_begin_index,\
                            'end_index':_last_head_end_index,'id': "generate_later"}) 
        elif e.tag == 'div' and e.get('type') == 'chapter':
            _last_chapter_end_index = len(all_textelements)-1
            all_annotations.append({'label':'chapter','begin_index': _last_chapter_begin_index,\
                            'end_index':_last_chapter_end_index,'id':'generate_later'})            
        elif e.tag == 'div' and e.get('type') == 'section':
            _last_section_end_index = len(all_textelements)-1
            all_annotations.append({'label':'section','begin_index': _last_section_begin_index,\
                            'end_index':_last_section_end_index,'id':'generate_later'})               
        elif e.tag == 'pb':
            # first store the 'previous' page, then store begin and end of currently closed page
            all_annotations.append({'label':'page','begin_index': _last_page_begin_index,\
                            'end_index':_last_page_end_index,'id': _last_page_id}) 
            _last_page_begin_index = _last_page_end_index
            _last_page_end_index = len(all_textelements)-1 
            _last_page_id = f"page-{e.get('n')}"
            
    return        

# use iterparse to traverse the xml hierarchy, depth first, post order
context = etree.iterparse(path, events=('start','end'))
for action, elem in context:
    handle_element(action,elem)
    

In [2]:
all_textelements[200:215]

['',
 'wyatt',
 '',
 '207',
 '',
 '',
 '',
 'HOOFDSTUK XIV.',
 '',
 '',
 '',
 '',
 'gevangenschap',
 '',
 '221']

In [3]:
len(all_textelements)

3130

In [4]:
all_annotations

[{'label': 'paragraph',
  'begin_index': 0,
  'end_index': 0,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 1,
  'end_index': 1,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 2,
  'end_index': 2,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 3,
  'end_index': 3,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 4,
  'end_index': 14,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 15,
  'end_index': 15,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 16,
  'end_index': 16,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 17,
  'end_index': 17,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 18,
  'end_index': 18,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 19,
  'end_index': 21,
  'id': 'generate_later'},
 {'label': 'paragraph',
  'begin_index': 22,
  'end_index': 22,
  'id': 'generate_later'},
 {'label

In [5]:
def get_annotations_overlapping_with(begin,end,annotations):
    return (a for a in annotations if (a['begin_index'] >= begin and a['begin_index'] < end) or\
           (a['end_index'] > begin and a['end_index'] <= end) or\
           (a['begin_index'] <= begin and a['end_index'] >= end))

In [6]:
# print all annotations for a chapter
for a in get_annotations_overlapping_with(636,832,all_annotations):
    print(a)

{'label': 'head', 'begin_index': 636, 'end_index': 636, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 637, 'end_index': 638, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 639, 'end_index': 639, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 640, 'end_index': 640, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 641, 'end_index': 641, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 642, 'end_index': 642, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 643, 'end_index': 643, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 644, 'end_index': 644, 'id': 'generate_later'}
{'label': 'page', 'begin_index': 635, 'end_index': 638, 'id': 'page-57'}
{'label': 'paragraph', 'begin_index': 645, 'end_index': 645, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 646, 'end_index': 646, 'id': 'generate_later'}
{'label': 'paragraph', 'begin_index': 647, 'end_index': 647, 'id': 'generate_later

In [7]:
# return generator for annotations with a specific label
def get_annotations_of_type(type,annotations):
    return (a for a in annotations if a['label'] == type)

In [8]:
# nested generators
def get_annotations_of_type_overlapping(type,begin,end,annotations):
    return get_annotations_of_type(type,(get_annotations_overlapping_with(begin,end,annotations)))

Variations of the above 3 functions, now for multiple types iso for just one type.

In [9]:
def get_annotations_of_types(types,annotations):
    return (a for a in annotations if a['label'] in types)

In [10]:
def get_annotations_of_types_overlapping(types,begin,end,annotations):
    return get_annotations_of_types(types,(get_annotations_overlapping_with(begin,end,annotations)))

In [11]:
def get_textlines_between(begin,end,annotations): 
    textlines = []
    types = ('paragraph', 'head')
    for line_annot in get_annotations_of_types_overlapping(types,begin,end,annotations):
        textlines.append((line_annot['begin_index'],line_annot['end_index'], \
                          all_textelements[line_annot['begin_index']:line_annot['end_index']]))
    return textlines

In [12]:
# get text for (text segment of) a chapter
for a in get_textlines_between(636,832,all_annotations):
    print(a)

(636, 636, [])
(637, 638, ["Met eene snelheid als die van het licht had zich het gerucht door gansch Londen verspreid, dat de edele Graaf van Devonshire met uitstekende eer ten hove ontvangen, in zijne rechtmatige bezittingen hersteld geworden en met een aantal gunsten overladen was. Bij de verwardheid van zoodanige volksgeruchten ontbrak het ook niet aan de noodige ongerijmdheden en overdrijvingen. Doch hoe uiteenloopend de lezingen over zijn plechtig onthaal op Whitehall ook mochten zijn, daarin waren allen, protestanten zoowel als katholieken, burgers zoowel als krijgsknechten, van den Lord-Mayor en zijne aldermen tot de schamele bewoners der afgelegenste dwarsstegen, het eens, dat dezen edelman geene zoo hooge eer kon bewezen worden, die zij allen hem niet waardig keurden. Ook verdrong men zich op die plaatsen, waar men hem nog dien eigen dag meende te zullen zien. Eene samengeschoolde volksmenigte leverde toenmaals, behoudens eenige karakteristieke uitzonderingen van tijd en volks

Gebruik bovenstaande voor het visualiseren van alle tekst voor een pagina, of alle heads (vanaf het midden van het boek)

In [13]:
for a in get_annotations_of_type('page', all_annotations):
    if a['id'] == 'page-116':
        print(a)

{'label': 'page', 'begin_index': 1036, 'end_index': 1042, 'id': 'page-116'}


In [14]:
for tl in get_textlines_between(1036,1042,all_annotations):
    print(tl)

(1037, 1037, [])
(1038, 1038, [])
(1039, 1039, [])
(1040, 1040, [])
(1041, 1042, ['Nog slechts ten deele hersteld van de verschillende aandoeningen, die zijn gemoed geschokt hadden, liep de Graaf van Devonshire eene der gewone hofzalen binnen. Men zegt hem dat de Koningin naar hem gevraagd heeft; men wijst hem het vertrek aan waar zij zich bevindt; hij begeeft zich daarheen; hij ziet er den Kanselier en Benefield bij haar. Eensklaps is het hem duidelijk, dat dit tot eene vreeselijke ontknooping leiden kan, dat beiden met elkander in betrekking kunnen staan en het eens tot zijn verderf. Als wij een belangrijk geheim met ons omgedragen, is het ons alsof elk onzer bewegingen dat'])


ISSUE: bovenstaande haalt alle overlappende paragrafen en heads op, maar zou moeten afbreken bij de page breaks die binnen paragrafen optreden. De kleinste basissegmenten zouden moeten worden opgehaald. Wat ontbreekt, is het equivalent van de lines, zoals in de republic casus.

Wat wel moet gebeuren: voor een end of paragraph moeten niet alle textonderdelen worden gejoined, maar allemaal afzonderlijk worden toegevoegd aan all_textelements. Vervolgens moet een paragraph annotation worden aangemaakt over al deze onderdelen.

In [15]:
for t in all_textelements[1036:1042]:
    print(f"{t}\n")

dedigen. Het ware hem wellust geweest, slechts eens den zoeten naam der geliefde voor een vertrouwd oor te mogen uitspreken. En toch moest hij laaghartig en zelfzuchtig schijnen, en toch gevoelde hij dat de ander niet meer gelooven kon aan de goedheid van zijn hart, toen hij antwoordde, zooals zijn gegeven woord het eischte:

‘Neen, Arundel, neen! ik kan haar niet helpen; voor haar vermag ik niets; vooralsnog niets;... misschien in het vervolg;... wie weet.... geloof mij, mijn vriend, laat ons geduld hebben.’

‘Dus heeft de beklagenswaardige ook u verloren,’ hervatte de Groot-Kamerheer koel en met stugheid. ‘Ik moet u nog met eene vraag lastig vallen, Mylord; het is de laatste over dit onderwerp. Zult gij mij tegen zijn, zoo ik in haar voordeel spreek?’

‘Op mijn ridderwoord, neen!’ riep Devonshire schielijk; zijne oogen schitterden van vreugde over eene uitkomst, die zoozeer met zijne wenschen instemde en hij stond op en vatte Arundel bij de hand. Zoo brandend jaagde het bloed hem doo

In [16]:
for a in get_annotations_of_type('page', all_annotations):
    print(a)

{'label': 'page', 'begin_index': 0, 'end_index': 295, 'id': ''}
{'label': 'page', 'begin_index': 295, 'end_index': 297, 'id': 'page-IV'}
{'label': 'page', 'begin_index': 297, 'end_index': 297, 'id': 'page-1'}
{'label': 'page', 'begin_index': 297, 'end_index': 298, 'id': 'page-3'}
{'label': 'page', 'begin_index': 298, 'end_index': 302, 'id': 'page-4'}
{'label': 'page', 'begin_index': 302, 'end_index': 315, 'id': 'page-5'}
{'label': 'page', 'begin_index': 315, 'end_index': 319, 'id': 'page-6'}
{'label': 'page', 'begin_index': 319, 'end_index': 325, 'id': 'page-7'}
{'label': 'page', 'begin_index': 325, 'end_index': 331, 'id': 'page-8'}
{'label': 'page', 'begin_index': 331, 'end_index': 334, 'id': 'page-9'}
{'label': 'page', 'begin_index': 334, 'end_index': 344, 'id': 'page-10'}
{'label': 'page', 'begin_index': 344, 'end_index': 350, 'id': 'page-11'}
{'label': 'page', 'begin_index': 350, 'end_index': 350, 'id': 'page-12'}
{'label': 'page', 'begin_index': 350, 'end_index': 350, 'id': 'page-

In [17]:
import json

# write all_textlines to a file
with open('../data/all_tei_textlines.txt', 'w') as filehandle:
    json.dump(all_textelements, filehandle)

In [18]:
for a in get_annotations_of_type('page', all_annotations):
    print(json.dumps(a, sort_keys=False, indent=2))

{
  "label": "page",
  "begin_index": 0,
  "end_index": 295,
  "id": ""
}
{
  "label": "page",
  "begin_index": 295,
  "end_index": 297,
  "id": "page-IV"
}
{
  "label": "page",
  "begin_index": 297,
  "end_index": 297,
  "id": "page-1"
}
{
  "label": "page",
  "begin_index": 297,
  "end_index": 298,
  "id": "page-3"
}
{
  "label": "page",
  "begin_index": 298,
  "end_index": 302,
  "id": "page-4"
}
{
  "label": "page",
  "begin_index": 302,
  "end_index": 315,
  "id": "page-5"
}
{
  "label": "page",
  "begin_index": 315,
  "end_index": 319,
  "id": "page-6"
}
{
  "label": "page",
  "begin_index": 319,
  "end_index": 325,
  "id": "page-7"
}
{
  "label": "page",
  "begin_index": 325,
  "end_index": 331,
  "id": "page-8"
}
{
  "label": "page",
  "begin_index": 331,
  "end_index": 334,
  "id": "page-9"
}
{
  "label": "page",
  "begin_index": 334,
  "end_index": 344,
  "id": "page-10"
}
{
  "label": "page",
  "begin_index": 344,
  "end_index": 350,
  "id": "page-11"
}
{
  "label": "page",


In [19]:
for a in get_annotations_of_type('paragraph', all_annotations):
    print(json.dumps(a, sort_keys=False, indent=2))

{
  "label": "paragraph",
  "begin_index": 0,
  "end_index": 0,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 1,
  "end_index": 1,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2,
  "end_index": 2,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 3,
  "end_index": 3,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 4,
  "end_index": 14,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 15,
  "end_index": 15,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 16,
  "end_index": 16,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 17,
  "end_index": 17,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 18,
  "end_index": 18,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 19,
  "end_index": 21,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 22,
  "end_index": 22,
  "id": "gene

}
{
  "label": "paragraph",
  "begin_index": 2438,
  "end_index": 2438,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2439,
  "end_index": 2439,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2440,
  "end_index": 2440,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2441,
  "end_index": 2441,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2442,
  "end_index": 2442,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2443,
  "end_index": 2443,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2444,
  "end_index": 2444,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2445,
  "end_index": 2445,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2446,
  "end_index": 2446,
  "id": "generate_later"
}
{
  "label": "paragraph",
  "begin_index": 2447,
  "end_index": 2447,
  "id": "generate_later"
}
{
  "label": "paragraph",
  